In [1]:
import networkx as nx
from anytree import Node, RenderTree, PreOrderIter, PostOrderIter, findall
from src.v1tree.operatorsold import merge
from graph.graphview import digraph_repr, draw
from copy import deepcopy
from enum import Enum, auto

In [2]:
NodeTypes = Enum('NodeTypes', 'INTER END')
INTER = NodeTypes.INTER
END = NodeTypes.END

In [3]:
root = nx.DiGraph([('x_0', 'f_1'), ('f_1', 'x_2'), ('x_1', 'f_1'), ('x_2', 'f_2'), ('x_3', 'f_2'), ('x_3', 'f_1'), ('x_5', 'f_3'), 
                   ('x_1', 'f_3'), ('f_3', 'x_3'), ('x_3', 'r_1'), ('x_4', 'r_1'), ('x_1', 'f_4'), ('f_4', 'x_6')])

In [4]:
# only leafs are explicit
treeroot = Node('root')
f0 = Node('f_0', node_type=INTER, parent=treeroot)
f1 = Node('f_1', node_type=INTER, parent=treeroot)
f2 = Node('f_2', node_type=INTER, parent=treeroot)
f3 = Node('f_3', node_type=INTER, parent=treeroot)
f4 = Node('f_4', node_type=INTER, parent=treeroot)
r1 = Node('r_1', node_type=END, parent=treeroot)

In [5]:
ch1, ch2, treerootn, solvefortable = merge(root, treeroot, ['f_1','f_2'], 'm1', solvefor=[('x_1', 'r_1')])

In [6]:
for pre, _, node in RenderTree(treerootn):
    treestr = u"%s%s" % (pre, node.name)
    print(treestr.ljust(16))

root            
├── f_0         
├── f_3         
├── f_4         
└── m1          
    ├── f_1     
    ├── f_2     
    └── r_1     


In [48]:
def residualizecomp(treeroot, name):
    treeroot = deepcopy(treeroot)
    node_to_res = list(findall(treeroot, filter_=lambda node: node.name ==name))[0]
    node_to_res.node_type=END
    return treeroot

In [39]:
def residualize(ch1, ch2, treeroot, name):
    edges = [(fr,to) for (fr,to) in ch1.edges() if to != name and fr != name]
    edges += ch2.edges()
    treeroot = deepcopy(treeroot)
    removednode = list(findall(treeroot, filter_=lambda node: node.name ==name))[0]
    parent = removednode.parent
    parent.children = [elt for child in parent.children for elt in (
        [child] if child != removednode else child.children)]
    return nx.DiGraph(edges), treeroot

In [40]:
ch3, treerootn2 = residualize(ch1, ch2, treerootn, 'm1')

In [25]:
gfinal = nx.DiGraph(edges)

In [11]:
def circular_vars(graph):
    return {elt:'o' if elt in ['x_0','x_1','x_2','x_3','x_4','x_5', 'x_6'] else 's' for elt in graph.nodes()}

In [12]:
def drawfull(graph, prog='neato'):
    draw(graph, node_shape=circular_vars(graph), arc=0.1, prog=prog);

In [34]:
set(root.edges()) == set(edges)

True